In [2]:
# -*- coding: utf-8 -*-
'''
# An implementation of sequence to sequence learning for performing addition

Input: "535+61"
Output: "596"
Padding is handled by using a repeated sentinel character (space)

Input may optionally be reversed, shown to increase performance in many tasks in:
"Learning to Execute"
http://arxiv.org/abs/1410.4615
and
"Sequence to Sequence Learning with Neural Networks"
http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
Theoretically it introduces shorter term dependencies between source and target.

Two digits reversed:
+ One layer LSTM (128 HN), 5k training examples = 99% train/test accuracy in 55 epochs

Three digits reversed:
+ One layer LSTM (128 HN), 50k training examples = 99% train/test accuracy in 100 epochs

Four digits reversed:
+ One layer LSTM (128 HN), 400k training examples = 99% train/test accuracy in 20 epochs

Five digits reversed:
+ One layer LSTM (128 HN), 550k training examples = 99% train/test accuracy in 30 epochs
'''  # noqa

from __future__ import print_function
from keras.models import Sequential
from keras import layers
import numpy as np
from six.moves import range

Using TensorFlow backend.


In [3]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one-hot integer representation
    + Decode the one-hot or integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One-hot encode given string C.

        # Arguments
            C: string, to be encoded.
            num_rows: Number of rows in the returned one-hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        """Decode the given vector or 2D array to their character output.

        # Arguments
            x: A vector or a 2D array of probabilities or one-hot representations;
                or a vector of character indices (used with `calc_argmax=False`).
            calc_argmax: Whether to find the character index with maximum
                probability, defaults to `True`.
        """
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)

In [4]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [5]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
REVERSE = True

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

In [6]:
# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

In [7]:
questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [8]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)

Vectorization...


In [9]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


In [10]:
# Try replacing GRU, or SimpleRNN.
RNN = layers.LSTM
HIDDEN_SIZE = 128
BATCH_SIZE = 128
LAYERS = 1

In [11]:
print('Build model...')
model = Sequential()
# "Encode" the input sequence using an RNN, producing an output of HIDDEN_SIZE.
# Note: In a situation where your input sequences have a variable length,
# use input_shape=(None, num_feature).
model.add(RNN(HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
# As the decoder RNN's input, repeatedly provide with the last output of
# RNN for each time step. Repeat 'DIGITS + 1' times as that's the maximum
# length of output, e.g., when DIGITS=3, max output is 999+999=1998.
model.add(layers.RepeatVector(DIGITS + 1))
# The decoder RNN could be multiple layers stacked or a single layer.
for _ in range(LAYERS):
    # By setting return_sequences to True, return not only the last output but
    # all the outputs so far in the form of (num_samples, timesteps,
    # output_dim). This is necessary as TimeDistributed in the below expects
    # the first dimension to be the timesteps.
    model.add(RNN(HIDDEN_SIZE, return_sequences=True))

# Apply a dense layer to the every temporal slice of an input. For each of step
# of the output sequence, decide which character should be chosen.
model.add(layers.TimeDistributed(layers.Dense(len(chars), activation='softmax')))
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

W0722 12:27:13.544011  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.



Build model...


W0722 12:27:14.022901  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0722 12:27:14.120470  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0722 12:27:14.973714  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0722 12:27:15.003582  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               72192     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4, 128)            131584    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4, 12)             1548      
Total params: 205,324
Trainable params: 205,324
Non-trainable params: 0
_________________________________________________________________


In [12]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 200):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1


W0722 12:27:15.629316  1588 deprecation.py:323] From C:\Users\lilanqing\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0722 12:27:17.328616  1588 deprecation_wrapper.py:119] From C:\Users\lilanqing\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 18s 401us/step - loss: 1.8887 - acc: 0.3209 - val_loss: 1.7805 - val_acc: 0.3421
Q 31+964  T 995  ☒ 102 
Q 25+836  T 861  ☒ 122 
Q 76+616  T 692  ☒ 102 
Q 490+289 T 779  ☒ 102 
Q 130+591 T 721  ☒ 102 
Q 51+213  T 264  ☒ 321 
Q 988+739 T 1727 ☒ 1113
Q 551+8   T 559  ☒ 52  
Q 942+9   T 951  ☒ 102 
Q 542+0   T 542  ☒ 42  

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 16s 349us/step - loss: 1.7320 - acc: 0.3613 - val_loss: 1.6638 - val_acc: 0.3805
Q 870+162 T 1032 ☒ 101 
Q 22+56   T 78   ☒ 22  
Q 873+1   T 874  ☒ 101 
Q 435+749 T 1184 ☒ 102 
Q 879+5   T 884  ☒ 101 
Q 437+18  T 455  ☒ 300 
Q 97+749  T 846  ☒ 100 
Q 0+191   T 191  ☒ 121 
Q 305+83  T 388  ☒ 300 
Q 304+348 T 652  ☒ 500 

--------------------------------------------------
Iteration 3
Train on 45000 sam

45000/45000 [==============================] - 15s 342us/step - loss: 0.1725 - acc: 0.9634 - val_loss: 0.1675 - val_acc: 0.9587
Q 381+20  T 401  ☑ 401 
Q 91+253  T 344  ☑ 344 
Q 235+62  T 297  ☑ 297 
Q 578+6   T 584  ☑ 584 
Q 422+869 T 1291 ☒ 1391
Q 1+115   T 116  ☑ 116 
Q 325+670 T 995  ☑ 995 
Q 19+936  T 955  ☑ 955 
Q 958+71  T 1029 ☑ 1029
Q 5+733   T 738  ☑ 738 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 342us/step - loss: 0.1332 - acc: 0.9737 - val_loss: 0.1301 - val_acc: 0.9724
Q 501+68  T 569  ☒ 579 
Q 270+22  T 292  ☑ 292 
Q 895+39  T 934  ☑ 934 
Q 4+575   T 579  ☒ 589 
Q 636+6   T 642  ☑ 642 
Q 29+687  T 716  ☑ 716 
Q 24+99   T 123  ☑ 123 
Q 11+339  T 350  ☑ 350 
Q 44+301  T 345  ☑ 345 
Q 688+861 T 1549 ☑ 1549

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

45000/45000 [==============================] - 15s 344us/step - loss: 0.0248 - acc: 0.9951 - val_loss: 0.0175 - val_acc: 0.9974
Q 2+158   T 160  ☑ 160 
Q 1+554   T 555  ☑ 555 
Q 719+3   T 722  ☑ 722 
Q 217+71  T 288  ☑ 288 
Q 2+487   T 489  ☑ 489 
Q 305+3   T 308  ☑ 308 
Q 129+29  T 158  ☑ 158 
Q 673+82  T 755  ☑ 755 
Q 749+224 T 973  ☑ 973 
Q 998+902 T 1900 ☑ 1900

--------------------------------------------------
Iteration 30
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 15s 342us/step - loss: 0.0128 - acc: 0.9989 - val_loss: 0.0182 - val_acc: 0.9967
Q 763+44  T 807  ☑ 807 
Q 4+18    T 22   ☑ 22  
Q 879+5   T 884  ☑ 884 
Q 5+637   T 642  ☑ 642 
Q 28+497  T 525  ☑ 525 
Q 46+278  T 324  ☑ 324 
Q 854+943 T 1797 ☑ 1797
Q 720+35  T 755  ☑ 755 
Q 424+920 T 1344 ☑ 1344
Q 464+679 T 1143 ☑ 1143

--------------------------------------------------
Iteration 31
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [===

...

45000/45000 [==============================] - 20s 440us/step - loss: 2.9683e-04 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 0.9994
Q 53+231  T 284  ☑ 284 
Q 47+253  T 300  ☑ 300 
Q 792+95  T 887  ☑ 887 
Q 13+91   T 104  ☑ 104 
Q 878+344 T 1222 ☑ 1222
Q 49+60   T 109  ☑ 109 
Q 758+904 T 1662 ☑ 1662
Q 81+626  T 707  ☑ 707 
Q 658+18  T 676  ☑ 676 
Q 690+324 T 1014 ☑ 1014

--------------------------------------------------
Iteration 142
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 20s 439us/step - loss: 6.6050e-04 - acc: 0.9999 - val_loss: 0.0111 - val_acc: 0.9964
Q 169+80  T 249  ☑ 249 
Q 82+240  T 322  ☑ 322 
Q 598+451 T 1049 ☑ 1049
Q 96+855  T 951  ☑ 951 
Q 92+417  T 509  ☒ 609 
Q 8+185   T 193  ☑ 193 
Q 57+299  T 356  ☑ 356 
Q 636+95  T 731  ☑ 731 
Q 44+372  T 416  ☑ 416 
Q 99+5    T 104  ☑ 104 

--------------------------------------------------
Iteration 143
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 13s 296us/step - loss: 4.1061e-04 - acc: 1.0000 - val_loss: 0.0018 - val_acc: 0.9996
Q 50+651  T 701  ☑ 701 
Q 48+405  T 453  ☑ 453 
Q 270+83  T 353  ☑ 353 
Q 27+602  T 629  ☑ 629 
Q 66+588  T 654  ☑ 654 
Q 609+915 T 1524 ☑ 1524
Q 109+6   T 115  ☑ 115 
Q 304+348 T 652  ☑ 652 
Q 988+21  T 1009 ☑ 1009
Q 594+81  T 675  ☑ 675 

--------------------------------------------------
Iteration 156
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 292us/step - loss: 3.5089e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9997
Q 28+9    T 37   ☑ 37  
Q 66+23   T 89   ☑ 89  
Q 1+554   T 555  ☑ 555 
Q 7+587   T 594  ☑ 594 
Q 379+63  T 442  ☑ 442 
Q 577+233 T 810  ☑ 810 
Q 57+982  T 1039 ☑ 1039
Q 954+39  T 993  ☑ 993 
Q 98+863  T 961  ☑ 961 
Q 40+47   T 87   ☑ 87  

--------------------------------------------------
Iteration 157
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 13s 296us/step - loss: 1.9103e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9997
Q 822+11  T 833  ☑ 833 
Q 66+810  T 876  ☑ 876 
Q 40+69   T 109  ☑ 109 
Q 922+86  T 1008 ☑ 1008
Q 26+972  T 998  ☑ 998 
Q 40+766  T 806  ☑ 806 
Q 19+443  T 462  ☑ 462 
Q 106+59  T 165  ☑ 165 
Q 630+24  T 654  ☑ 654 
Q 57+906  T 963  ☑ 963 

--------------------------------------------------
Iteration 170
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 296us/step - loss: 0.0376 - acc: 0.9899 - val_loss: 0.0101 - val_acc: 0.9970
Q 713+209 T 922  ☑ 922 
Q 388+150 T 538  ☑ 538 
Q 22+979  T 1001 ☑ 1001
Q 21+82   T 103  ☑ 103 
Q 909+56  T 965  ☑ 965 
Q 374+3   T 377  ☑ 377 
Q 179+428 T 607  ☑ 607 
Q 673+170 T 843  ☑ 843 
Q 7+434   T 441  ☑ 441 
Q 8+423   T 431  ☑ 431 

--------------------------------------------------
Iteration 171
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/4500

45000/45000 [==============================] - 13s 296us/step - loss: 6.0407e-04 - acc: 1.0000 - val_loss: 0.0017 - val_acc: 0.9997
Q 76+954  T 1030 ☑ 1030
Q 33+168  T 201  ☑ 201 
Q 978+287 T 1265 ☑ 1265
Q 578+2   T 580  ☑ 580 
Q 85+110  T 195  ☑ 195 
Q 905+78  T 983  ☑ 983 
Q 315+548 T 863  ☑ 863 
Q 1+85    T 86   ☑ 86  
Q 652+39  T 691  ☑ 691 
Q 43+461  T 504  ☑ 504 

--------------------------------------------------
Iteration 184
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 298us/step - loss: 4.0233e-04 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 0.9997
Q 117+1   T 118  ☑ 118 
Q 467+526 T 993  ☑ 993 
Q 51+664  T 715  ☑ 715 
Q 736+864 T 1600 ☑ 1600
Q 940+95  T 1035 ☑ 1035
Q 4+852   T 856  ☑ 856 
Q 500+597 T 1097 ☑ 1097
Q 769+4   T 773  ☑ 773 
Q 17+677  T 694  ☑ 694 
Q 90+59   T 149  ☑ 149 

--------------------------------------------------
Iteration 185
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

45000/45000 [==============================] - 13s 295us/step - loss: 2.2602e-04 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 0.9998
Q 701+88  T 789  ☑ 789 
Q 431+637 T 1068 ☑ 1068
Q 76+160  T 236  ☑ 236 
Q 628+57  T 685  ☑ 685 
Q 363+968 T 1331 ☑ 1331
Q 53+576  T 629  ☑ 629 
Q 80+134  T 214  ☑ 214 
Q 605+81  T 686  ☑ 686 
Q 94+694  T 788  ☑ 788 
Q 485+756 T 1241 ☑ 1241

--------------------------------------------------
Iteration 198
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 13s 298us/step - loss: 1.9956e-04 - acc: 1.0000 - val_loss: 0.0013 - val_acc: 0.9998
Q 911+95  T 1006 ☑ 1006
Q 72+59   T 131  ☑ 131 
Q 93+52   T 145  ☑ 145 
Q 208+9   T 217  ☑ 217 
Q 277+949 T 1226 ☑ 1226
Q 308+470 T 778  ☑ 778 
Q 630+24  T 654  ☑ 654 
Q 478+845 T 1323 ☑ 1323
Q 742+291 T 1033 ☑ 1033
Q 268+216 T 484  ☑ 484 

--------------------------------------------------
Iteration 199
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/

In [14]:
import os
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'addition_rnn_trained_model.h5'

# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

Saved trained model at D:\lilanqing\CODE\Python_learn\keras-study\saved_models\addition_rnn_trained_model.h5 
